<a href="https://colab.research.google.com/github/IsaacFigNewton/DAG-Based-Compression/blob/main/DAG_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO



1.   Switch from dictionary representation of children to set
2.   Fix pruning


# Install packages and import libraries

In [1]:
# !pip install --upgrade ete3  # http://etetoolkit.org/docs/latest/reference/reference_treeview.html#treestyle
# !pip install PyQt5

# import os
# # Set the environment variables for PyQt5
# os.environ['QT_QPA_PLATFORM_PLUGIN_PATH'] = '/usr/lib/x86_64-linux-gnu/qt5/plugins'
# os.environ['QT_PLUGIN_PATH'] = '/usr/lib/x86_64-linux-gnu/qt5/plugins'
# os.environ['QT_QPA_PLATFORM'] = 'xcb'
# os.environ['QT_DEBUG_PLUGINS'] = '1'

# # Keep trying to get this working to make things look nice
# import ete3
# from ete3 import Tree, TreeStyle

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

import warnings
# warnings.filterwarnings('ignore')

# Classes

In [30]:
class SuffixNode:
    def __init__(self, suffix=None, frequency=0, children=None, parent=None):
        self.suffix = suffix
        self.frequency = frequency

        if children is None:
            children = set()  # Initialize children as an empty set
        self.children = children
        self.parent = parent

    def __str__(self):
        return self.suffix

    def add_suffix(self, suffix):
        if not suffix:
            return  # If the suffix is empty, return

        # Find the longest prefix match in the children
        for child in self.children:
            child_key = child.suffix

            # Find the index of the longest common prefix
            i = 0
            while i < len(child_key) and i < len(suffix) and child_key[i] == suffix[i]:
                i += 1

            # If there is a common prefix
            if i > 0:
                # Update the frequency of the current node
                child.frequency += 1

                # If the common prefix is the entire child key, recurse into that child
                if i == len(child_key):
                    child.add_suffix(suffix[i:])

                # If the common prefix is only part of the child key, split the edge
                elif i < len(suffix):
                    old_suffix = child_key[i:]
                    new_suffix = suffix[i:]

                    # Replace the old entry for the current node with a new one for the edge split
                    new_child = SuffixNode(suffix=child_key[:i],
                                           frequency=child.frequency,
                                           parent=self)

                    # Transfer the children to the new split node
                    new_child.children = child.children
                    for grandchild in new_child.children:
                        grandchild.parent = new_child

                    # Remove the old child and add the new split node
                    self.children.remove(child)
                    self.children.add(new_child)

                    # Update the current node
                    child = new_child

                    # Create a new node for the existing edge suffix
                    split_node = SuffixNode(suffix=old_suffix,
                                            frequency=1,
                                            parent=child)
                    split_node.children = child.children

                    # Create a new node for the new suffix
                    new_child.children = set()
                    new_child.children.add(split_node)
                    new_child.children.add(SuffixNode(suffix=new_suffix,
                                                      frequency=1,
                                                      parent=child))
                return

        # No matching prefix, add the suffix as a new child
        self.children.add(SuffixNode(suffix=suffix,
                                     frequency=1,
                                     parent=self))

In [31]:
def print_tree(tree, indent = 0):
    # Iterate over the child.suffixs (features) in the tree
    for child in tree.children:
        print(' ' * indent + str(child.suffix) + ": " + str(child.frequency))
        # If the child is a SuffixNode, recursively print the subtree
        if isinstance(child, SuffixNode):
            print_tree(child, indent + 4)
        else:
            print(' ' * (indent + 4) + str(child.suffix) + ": " + str(child.frequency))

#Build Suffix Tree

In [32]:
test = "CAGTCAGG"

root = SuffixNode()
root = SuffixNode(children = {
    SuffixNode(suffix="C"),
    SuffixNode(suffix="A"),
    SuffixNode(suffix="T"),
    SuffixNode(suffix="G")
})

# loop through the string, starting with the last character
for i in range(0, len(test)):
  suffix = test[len(test) - i - 1:]

  # add the suffix to the tree
  root.add_suffix(suffix)

In [34]:
print_tree(root)

T: 1
    CAGG: 1
C: 2
    AG: 2
        TCAGG: 1
        G: 1
G: 3
    TCAGG: 1
    G: 1
A: 2
    G: 2
        G: 1
        TCAGG: 1


In [ ]:
# root.prune_tree(1)

# To fix later

In [ ]:
# # Loads a tree structure from a newick string. The returned variable ’t’ is the root node for the tree.
# t = Tree("(A:1,(B:1,(E:1,D:1):0.5):0.5);" )

# print(t)

# # ts = TreeStyle()
# # ts.show_leaf_name = True

# # t.show(tree_style=ts)